In [ ]:
import arcpy
import os
import shutil
from arcpy.sa import Raster, SetNull

# -----------------------------
# User settings
# -----------------------------
arcpy.env.overwriteOutput = True
parent_folder = r"C:\Users\ss2596\Downloads\Mufunta_Sentinel2_aug_2023"
threshold = 1000  # pixels below this are masked
use_histogram_matching = True   # apply if Image Analyst available

# -----------------------------
# Prepare temporary folder
# -----------------------------
temp_folder = os.path.join(parent_folder, "temp_tiffs")
os.makedirs(temp_folder, exist_ok=True)

# -----------------------------
# Check Image Analyst extension
# -----------------------------
ia_available = False
if use_histogram_matching:
    try:
        if arcpy.CheckExtension("ImageAnalyst") == "Available":
            arcpy.CheckOutExtension("ImageAnalyst")
            from arcpy.ia import HistogramMatch
            ia_available = True
            print("✅ Image Analyst extension checked out — histogram matching enabled.")
        else:
            print("⚠️ Image Analyst extension not available — skipping histogram matching.")
    except Exception as e:
        print(f"⚠️ Could not check Image Analyst extension — skipping histogram matching. ({e})")

# -----------------------------
# Sentinel-2 bands to process
# -----------------------------
bands = {"B02": "Blue", "B03": "Green", "B04": "Red"}
mosaicked_band_files = []

# -----------------------------
# Find SAFE folders
# -----------------------------
safe_folders = sorted([f for f in os.listdir(parent_folder) if f.endswith(".SAFE")])
print(f"Found {len(safe_folders)} SAFE folders.")

# -----------------------------
# Process each band
# -----------------------------
for band_code, band_name in bands.items():
    print(f"\nProcessing {band_name} ({band_code})...")
    tiff_list = []
    counter = 0

    # Extract and mask each JP2
    for safe in safe_folders:
        granule_path = os.path.join(parent_folder, safe, "GRANULE")
        if not os.path.exists(granule_path):
            continue
        for granule_id in os.listdir(granule_path):
            img_data_path = os.path.join(granule_path, granule_id, "IMG_DATA", "R10m")
            if not os.path.exists(img_data_path):
                continue
            for f in os.listdir(img_data_path):
                if band_code in f and f.endswith(".jp2") and not f.startswith("MSK_"):
                    counter += 1
                    jp2_path = os.path.join(img_data_path, f)
                    tiff_path = os.path.join(temp_folder, f"{os.path.splitext(f)[0]}_masked.tif")

                    # Copy JP2 to TIFF and mask low values
                    band_r = Raster(jp2_path)
                    masked_r = SetNull((band_r < threshold) | (band_r == 0), band_r)
                    masked_r.save(tiff_path)
                    tiff_list.append(tiff_path)
                    print(f"  [{counter}] Masked: {f}")

    if not tiff_list:
        print(f"⚠️ No valid rasters found for {band_name}, skipping.")
        continue

    # -----------------------------
    # Histogram matching (optional)
    # -----------------------------
    if use_histogram_matching and ia_available and len(tiff_list) > 1:
        print(f"  Performing histogram matching for {band_name}...")
        ref_raster = tiff_list[0]
        normalized_list = [ref_raster]
        for r in tiff_list[1:]:
            norm_path = r.replace("_masked.tif", "_norm.tif")
            HistogramMatch(in_raster=r, target_raster=ref_raster, out_raster=norm_path)
            normalized_list.append(norm_path)
    else:
        if use_histogram_matching and not ia_available:
            print(f"  Skipping histogram matching for {band_name} (extension not available).")
        normalized_list = tiff_list

    # -----------------------------
    # Mosaic band
    # -----------------------------
    out_raster = os.path.join(temp_folder, f"mosaic_{band_code}.tif")
    print(f"  Mosaicking {len(normalized_list)} rasters for {band_name} into {out_raster}...")
    if len(normalized_list) > 1:
        arcpy.management.CopyRaster(normalized_list[0], out_raster)
        arcpy.management.Mosaic(
            inputs=normalized_list[1:],
            target=out_raster,
            mosaic_type="BLEND",
            nodata_value="0"
        )
    else:
        arcpy.management.CopyRaster(normalized_list[0], out_raster)

    mosaicked_band_files.append(out_raster)
    print(f"  ✔ {band_name} mosaic saved: {out_raster}")

# -----------------------------
# Combine into final RGB
# -----------------------------
if mosaicked_band_files and len(mosaicked_band_files) == 3:
    rgb_order = [
        os.path.join(temp_folder, "mosaic_B04.tif"),  # Red
        os.path.join(temp_folder, "mosaic_B03.tif"),  # Green
        os.path.join(temp_folder, "mosaic_B02.tif")   # Blue
    ]
    folder_name = os.path.basename(parent_folder.rstrip("\\/"))
    final_rgb = os.path.join(parent_folder, f"{folder_name}.tif")
    print(f"\nCreating final RGB mosaic: {final_rgb}...")
    arcpy.management.CompositeBands(rgb_order, final_rgb)
    arcpy.management.CalculateStatistics(final_rgb)
    arcpy.management.BuildPyramids(final_rgb)
    print(f"✅ Final RGB mosaic created: {final_rgb}")
else:
    print("❌ Not all bands were processed, RGB mosaic not created.")

# -----------------------------
# Cleanup
# -----------------------------
print(f"\nCleaning up temporary folder: {temp_folder}")
try:
    shutil.rmtree(temp_folder)
    print("✅ Temporary files removed.")
except Exception as e:
    print(f"⚠️ Could not remove temp folder: {e}")


⚠️ Could not check Image Analyst extension — skipping histogram matching. (cannot import name 'HistogramMatch' from 'arcpy.ia' (C:\Program Files\ArcGIS\Pro\Resources\ArcPy\arcpy\ia\__init__.py))
Found 6 SAFE folders.

Processing Blue (B02)...
  [1] Masked: T35LKD_20230823T081609_B02_10m.jp2
  [2] Masked: T35LKC_20230823T081609_B02_10m.jp2
  [3] Masked: T35LLD_20230823T081609_B02_10m.jp2
  [4] Masked: T35LLC_20230823T081609_B02_10m.jp2
  [5] Masked: T35LLC_20230825T080611_B02_10m.jp2
  [6] Masked: T35LLD_20230825T080611_B02_10m.jp2
  Skipping histogram matching for Blue (extension not available).
  Mosaicking 6 rasters for Blue into C:\Users\ss2596\Downloads\Mufunta_Sentinel2_aug_2023\temp_tiffs\mosaic_B02.tif...
  ✔ Blue mosaic saved: C:\Users\ss2596\Downloads\Mufunta_Sentinel2_aug_2023\temp_tiffs\mosaic_B02.tif

Processing Green (B03)...
  [1] Masked: T35LKD_20230823T081609_B03_10m.jp2
  [2] Masked: T35LKC_20230823T081609_B03_10m.jp2
  [3] Masked: T35LLD_20230823T081609_B03_10m.jp2
  